In [ ]:
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 

### 40. 係り受け解析結果の読み込み（形態素）

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [18]:
class Morph:
    def __init__(self, line):
        surface, other = line.split("\t")
        other = other.split(",")
        self.surface = surface
        self.base = other[-3]
        self.pos = other[0]
        self.pos1 = other[1]

sentences = [] #文リスト
morphs = [] #形態素リスト

with open("./ai.ja.txt.parsed") as f:
    for line in f:
        if line[0] == "*":
            continue
        elif line != "EOS\n": 
            morphs.append(Morph(line))
        else:  #EOS（文末）の場合
            sentences.append(morphs)
            morphs = []

for i in sentences[0]:
    print(vars(i))


{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}


### 41. 係り受け解析結果の読み込み（文節・係り受け）

40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [19]:
class Sentence:
    def __init__(self, chunks):
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks):
            if chunk.dst not in [None, -1]:
                self.chunks[chunk.dst].srcs.append(i)

class Chunk:
    def __init__(self, morphs, dst, chunk_id):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []
        self.chunk_id = chunk_id

class Morph:
    def __init__(self, line):
        surface, other = line.split("\t")
        other = other.split(",")
        self.surface = surface
        self.base = other[-3]
        self.pos = other[0]
        self.pos1 = other[1]

sentences = [] #文リスト
chunks = [] #節リスト
morphs = [] #形態素リスト
chunk_id = 0 #文節番号

with open("./ai.ja.txt.parsed") as f:
    for line in f:
        if line[0] == "*":
            if morphs:
                chunks.append(Chunk(morphs, dst, chunk_id))
                chunk_id += 1
                morphs = []
            dst = int(line.split()[2].replace("D", ""))
        elif line != "EOS\n":
            morphs.append(Morph(line))
        else:
            chunks.append(Chunk(morphs, dst, chunk_id))
            sentences.append(Sentence(chunks))
        
            morphs = []
            chunks = []
            dst = None
            chunk_id = 0

for chunk in sentences[2].chunks:
    chunk_str = "".join([morph.surface for morph in chunk.morphs])
    print(f"文節の文字列：{chunk_str}\n係り先の文節番号：{chunk.dst}\n")


文節の文字列：人工知能
係り先の文節番号：17

文節の文字列：（じんこうちのう、、
係り先の文節番号：17

文節の文字列：AI
係り先の文節番号：3

文節の文字列：〈エーアイ〉）とは、
係り先の文節番号：17

文節の文字列：「『計算
係り先の文節番号：5

文節の文字列：（）』という
係り先の文節番号：9

文節の文字列：概念と
係り先の文節番号：9

文節の文字列：『コンピュータ
係り先の文節番号：8

文節の文字列：（）』という
係り先の文節番号：9

文節の文字列：道具を
係り先の文節番号：10

文節の文字列：用いて
係り先の文節番号：12

文節の文字列：『知能』を
係り先の文節番号：12

文節の文字列：研究する
係り先の文節番号：13

文節の文字列：計算機科学
係り先の文節番号：14

文節の文字列：（）の
係り先の文節番号：15

文節の文字列：一分野」を
係り先の文節番号：16

文節の文字列：指す
係り先の文節番号：17

文節の文字列：語。
係り先の文節番号：34

文節の文字列：「言語の
係り先の文節番号：20

文節の文字列：理解や
係り先の文節番号：20

文節の文字列：推論、
係り先の文節番号：21

文節の文字列：問題解決などの
係り先の文節番号：22

文節の文字列：知的行動を
係り先の文節番号：24

文節の文字列：人間に
係り先の文節番号：24

文節の文字列：代わって
係り先の文節番号：26

文節の文字列：コンピューターに
係り先の文節番号：26

文節の文字列：行わせる
係り先の文節番号：27

文節の文字列：技術」、または、
係り先の文節番号：34

文節の文字列：「計算機
係り先の文節番号：29

文節の文字列：（コンピュータ）による
係り先の文節番号：31

文節の文字列：知的な
係り先の文節番号：31

文節の文字列：情報処理システムの
係り先の文節番号：33

文節の文字列：設計や
係り先の文節番号：33

文節の文字列：実現に関する
係り先の文節番号：34

文節の文字列：研究分野」とも
係り先の文節番号：35

文節の文字列：される。
係り先の文節番号：-1



### 42. 係り元と係り先の文節の表示Permalink

係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [20]:
for chunk in sentences[2].chunks:
    if int(chunk.dst) == -1:
        continue
    else:
        surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
        next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"]) 
        print(f"{surf}\t{next_surf}")


人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される


### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [21]:
for chunk in sentences[2].chunks:
    if int(chunk.dst) == -1:
        continue
    else:
        surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
        next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"]) 
        pos_noun = [morph.surface for morph in chunk.morphs if morph.pos == "名詞"]
        pos_verb = [morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos == "動詞"]
        if pos_noun and pos_verb: 
            print(f"{surf}\t{next_surf}")


道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される


### 44. 係り受け木の可視化Permalink

与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [22]:
import pydot_ng as pydot

pair = []
for chunk in sentences[2].chunks:
    if int(chunk.dst) == -1:
        continue
    else:
        surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
        next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"]) #文節のリストに係り先番号をindexに指定。その文節の形態素リストを取得
        pair.append((surf, next_surf))

img = pydot.Dot()
img.set_node_defaults(fontname="MS Mincho")
for s, t in pair:
    img.add_edge(pydot.Edge(s, t))
img.write_png("./result44.png")


True

### 45. 動詞の格パターンの抽出

今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
```
作り出す	で は を
```
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ
- 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [23]:
with open("./result45.txt", "w") as f:
    for i in range(len(sentences)):
        for chunk in sentences[i].chunks:
            for morph in chunk.morphs:
                if morph.pos == "動詞": 
                    particles = []
                    for src in chunk.srcs:
                        particles += [morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos == "助詞"]
                    if len(particles) > 1:
                        particles = set(particles)
                        particles = sorted(list(particles))
                        form = " ".join(particles)
                        print(f"{morph.base}\t{form}", file=f)


### 46. 動詞の格フレーム情報の抽出

45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
```
作り出す	で は を	会議で ジョンマッカーシーは 用語を
```

In [24]:
with open("./result46.txt", "w") as f:
    for i in range(len(sentences)):
        for chunk in sentences[i].chunks:
            for morph in chunk.morphs:
                if morph.pos == "動詞": 
                    particles = []
                    items = []
                    for src in chunk.srcs:
                        particles += [morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos == "助詞"]
                        items += ["".join([morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos != "記号"])]
                    if len(particles) > 1:
                        if len(items) > 1:
                            particles = sorted(set(particles))
                            items = sorted(set(items))
                            particles_form = " ".join(particles)
                            items_form = " ".join(items)
                            print(f"{morph.base}\t{particles_form}\t{items_form}", file=f)


### 47. 機能動詞構文のマイニング

動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．
```
学習を行う	に を	元に 経験を
```

In [25]:
with open("./result47.txt", "w") as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            for morph in chunk.morphs:
                if morph.pos == "動詞": 
                    for src in chunk.srcs:
                        predicates = []
                        if len(sentence.chunks[src].morphs) == 2 and sentence.chunks[src].morphs[0].pos1 == "サ変接続" and sentence.chunks[src].morphs[1].surface == "を":
                            predicates = "".join([sentence.chunks[src].morphs[0].surface, sentence.chunks[src].morphs[1].surface, morph.base])
                            particles = []
                            items = []
                            for src in chunk.srcs:
                                particles += [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == "助詞"]
                                item = "".join([morph.surface for morph in sentence.chunks[src].morphs if morph.pos != "記号"])
                                item = item.rstrip()
                                if item not in predicates:
                                    items.append(item)
                            if len(particles) > 1:
                                if len(items) > 1:
                                    particles = sorted(set(particles))
                                    items = sorted(set(items))
                                    particles_form = " ".join(particles)
                                    items_form = " ".join(items)
                                    predicate = " ".join(predicates)
                                    print(f"{predicates}\t{particles_form}\t{items_form}", file=f)


### 48. 名詞から根へのパスの抽出

文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した
```
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した
```

In [26]:
sentence = sentences[2]
for chunk in sentence.chunks:
    for morph in chunk.morphs:
        if "名詞" in morph.pos:
            path = ["".join(morph.surface for morph in chunk.morphs if morph.pos != "記号")]
            while chunk.dst != -1:
                path.append("".join(morph.surface for morph in sentence.chunks[chunk.dst].morphs if morph.pos != "記号"))
                chunk = sentence.chunks[chunk.dst]
            print("->".join(path))


人工知能->語->研究分野とも->される
される
じんこうちのう->語->研究分野とも->される
される
AI->エーアイとは->語->研究分野とも->される
エーアイとは->語->研究分野とも->される
計算->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
概念と->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
コンピュータ->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
知能を->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
計算機科学->の->一分野を->指す->語->研究分野とも->される
される
される
一分野を->指す->語->研究分野とも->される
される
語->研究分野とも->される
言語の->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
理解や->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
される
知的行動を->代わって->行わせる->技術または->研究分野とも->される
される
人間に->代わって->行わせる->技術または->研究分野とも->される
コンピューターに->行わせる->技術または->研究分野とも->される
技術または->研究分野とも->される
計算機->コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
される
コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
知的な->情報処理システムの->実現に関する->研究分野とも->される
情報処理システムの->実現に関する

### 49. 名詞間の係り受けパスの抽出Permalink

文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i < j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
```
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを
```

In [27]:
from itertools import combinations
import re

sentence = sentences[2]
nouns = []
for i, chunk in enumerate(sentence.chunks):
    if [morph for morph in chunk.morphs if morph.pos == "名詞"]:
        nouns.append(i)
for i, j in combinations(nouns, 2):
    path_I = []
    path_J = []
    while i != j:
        if i < j: #文節iの構文木経路上に文節jが存在する場合
            path_I.append(i)
            i = sentence.chunks[i].dst
        else: #文節iの構文木経路上に文節jがない場合
            path_J.append(j)
            j = sentence.chunks[j].dst
    
    if len(path_J) == 0: # 文節Iの構文木上に文節Jが存在する場合
        X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        Y = "Y" +  "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "名詞" and morph.pos != "記号"])
        chunk_X = re.sub("X+", "X", X)
        chunk_Y = re.sub("Y+", "Y", Y)
        path_ItoJ = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs)] + [chunk_Y]
        print(" -> ".join(path_ItoJ))
    else: # 文節Iの構文木上に文節Jが存在しない場合
        X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        Y = "Y" + "".join([morph.surface for morph in sentence.chunks[path_J[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"]) 
        chunk_X = re.sub("X+", "X", X)
        chunk_Y = re.sub("Y+", "Y", Y)
        chunk_k = "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "記号"])
        path_X = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
        path_Y = [chunk_Y] + ["".join(morph.surface for n in path_J[1: ]for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
        print(" | ".join(["->".join(path_X), "->".join(path_Y), chunk_k]))


X-> | Yのう-> | 語
X-> | Y->エーアイとは | 語
X-> | Yとは-> | 語
X-> | Y->という道具を用いて研究する計算機科学の一分野を指す | 語
X-> | Yと->道具を用いて研究する計算機科学の一分野を指す | 語
X-> | Y->という道具を用いて研究する計算機科学の一分野を指す | 語
X-> | Yを->用いて研究する計算機科学の一分野を指す | 語
X-> | Yを->研究する計算機科学の一分野を指す | 語
X-> | Yする->計算機科学の一分野を指す | 語
X-> | Y->の一分野を指す | 語
X-> | Yを->指す | 語
X ->  -> Y
X->語 | Yの->推論問題解決などの知的行動を代わって行わせる技術または | 研究分野とも
X->語 | Yや->推論問題解決などの知的行動を代わって行わせる技術または | 研究分野とも
X->語 | Y->問題解決などの知的行動を代わって行わせる技術または | 研究分野とも
X->語 | Yなどの->知的行動を代わって行わせる技術または | 研究分野とも
X->語 | Yを->代わって行わせる技術または | 研究分野とも
X->語 | Yに->代わって行わせる技術または | 研究分野とも
X->語 | Yに->行わせる技術または | 研究分野とも
X->語 | Yまたは-> | 研究分野とも
X->語 | Y->コンピュータによる情報処理システムの実現に関する | 研究分野とも
X->語 | Yによる->情報処理システムの実現に関する | 研究分野とも
X->語 | Yな->情報処理システムの実現に関する | 研究分野とも
X->語 | Yの->実現に関する | 研究分野とも
X->語 | Yや->実現に関する | 研究分野とも
X->語 | Yに関する-> | 研究分野とも
X -> 語。 -> Yとも
Xのう-> | Y->エーアイとは | 語
Xのう-> | Yとは-> | 語
Xのう-> | Y->という道具を用いて研究する計算機科学の一分野を指す | 語
Xのう-> | Yと->道具を用いて研究する計算機科学の一分野を指す | 語
Xのう-> | Y->という道具を用いて研究する計算機科学の一分野を指す | 語
Xのう-> | Yを->用いて研